## Read the Data

Import necessary libraries.

In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from plotly import graph_objs as go

Read the data.

In [2]:
data = None

with open('new_data/300wLP_anno_tr.json') as f:
    ignore = ['isValidation', 'img_width', 'img_height']
    
    data = pd.read_json(f)
    data = data.drop(columns=ignore)

In [3]:
data.head()

,img_paths,objpos,scale_provided,bbox,landmarks,landmarks_2d,head_pose
0,AFW/AFW_1051618982_1_0.jpg,"[227.0259552, 298.4931946]",0.911768,"[135.8491516, 215.2526703, 182.3536072, 166.48...","[[135.5419922, 243.8638153, -67.62865448], [13...","[[135.8491516, 242.7403412], [137.8736725, 266...","[-5.037350655, 0.2395197004, 1.523025155]"
1,AFW/AFW_1051618982_1_1.jpg,"[231.3812561, 299.8146362]",0.908580,"[140.5232239, 216.4512634, 181.7160645, 166.72...","[[140.4693909, 240.2672119, -73.68688965], [14...","[[140.5232239, 239.2648773], [140.6314392, 266...","[-7.089375019, 5.239519596, 2.212796926]"
2,AFW/AFW_1051618982_1_10.jpg,"[235.726532, 295.34552]",0.853296,"[150.3969269, 210.1190948, 170.6592255, 170.45...","[[204.7253418, 234.839035, -117.61026], [203.4...","[[157.4411926, 245.7213745], [150.9309692, 268...","[-7.032663822, 50.23952103, 2.25369072]"
3,AFW/AFW_1051618982_1_11.jpg,"[231.4551697, 294.9733276]",0.854604,"[145.994812, 209.8023987, 170.9207153, 170.341...","[[213.2453308, 236.2964172, -119.8408813], [21...","[[158.3337402, 246.4070435], [152.5475311, 269...","[-6.688948631, 55.23952103, 1.5502201320000002]"
4,AFW/AFW_1051618982_1_12.jpg,"[226.9594116, 294.1109924]",0.855151,"[141.4443207, 208.5832672, 171.0301666, 171.05...","[[222.3459473, 234.3593445, -119.6676025], [22...","[[160.0543671, 245.4129791], [154.4994202, 269...","[-7.17603159, 60.23952484, 2.234272718]"


Filtered Data

In [4]:
# Get all the elements but the last one of the split.
data['base_img_paths'] = data['img_paths'].str.split('_').str[:-1].str.join('_')

# Create the new DataFrame by grouping the 2D landmarks.
lm = data.groupby('base_img_paths').agg({
    'landmarks': lambda x: np.array(x.tolist()),
    'landmarks_2d': lambda x: np.array(x.tolist())
}).reset_index()

Sub-Index Counts

In [5]:
lm['landmarks_2d'].apply(lambda x: x.shape[0]).value_counts()

landmarks_2d
18    1190
17     733
16     578
15     444
14     354
13     229
12     175
11      80
10      40
9       11
8        3
Name: count, dtype: int64

Save the data.

In [6]:
# Save the DataFrame "lm".
lm.to_json('new_data/lm_300wLP_anno_tr.json')

## Plot 3D

In [9]:
def plot_3d(row):
    
    landmarks_3d = np.array(row.landmarks[0]).transpose()
    
    fig = go.Figure()

    fig.add_trace(go.Scatter3d(
        x=landmarks_3d[0],
        y=landmarks_3d[1],
        z=landmarks_3d[2],
        mode='markers',
        marker=dict(size=3, color='red'),
        name='3D Landmarks'
    ))

    # Update layout for interactive controls
    fig.update_layout(
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z'),
            aspectmode='cube',  # Cube aspect ratio for equal scaling
            camera=dict(up=dict(x=0, y=0, z=1), eye=dict(x=0, y=0, z=-2)),  # Initial camera position
        ),
        title=row.base_img_paths
    )

    # Show the interactive plot in Jupyter Notebook or in a separate browser window
    fig.show()

    #plt.scatter(landmarks_2d[0], landmarks_2d[1], c='red', s=3)
    #plt.imshow(image)
    plt.show()

In [10]:
index = 4
row = lm.iloc[index]
plot_3d(row)